## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
print("Current working directory: {0}".format(os.getcwd()))
# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)


Current working directory: c:\Users\Jeremy\Desktop\Bootcamp\WeatherPy\Weather_Database


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("city_weather.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Decription,Date
0,Hithadhoo,-0.6000,73.0833,82.36,73,64,8.01,MV,broken clouds,2022-08-10 17:39:41
1,New Norfolk,-42.7826,147.0587,40.91,75,91,3.22,AU,overcast clouds,2022-08-10 17:29:51
2,Hobart,-42.8794,147.3294,41.27,82,95,6.91,AU,overcast clouds,2022-08-10 17:39:42
3,Matagami,49.7502,-77.6328,77.61,37,2,12.50,CA,clear sky,2022-08-10 17:39:42
4,Hermanus,-34.4187,19.2345,56.80,88,100,9.42,ZA,overcast clouds,2022-08-10 17:39:43


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_temperatures_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                          (city_data_df["Max Temp"] >= min_temp)]
preferred_temperatures_df.head(10)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Decription,Date
0,Hithadhoo,-0.6000,73.0833,82.36,73,64,8.01,MV,broken clouds,2022-08-10 17:39:41
3,Matagami,49.7502,-77.6328,77.61,37,2,12.50,CA,clear sky,2022-08-10 17:39:42
4,Hermanus,-34.4187,19.2345,56.80,88,100,9.42,ZA,overcast clouds,2022-08-10 17:39:43
6,Port Alfred,-33.5906,26.8910,61.83,82,87,4.68,ZA,overcast clouds,2022-08-10 17:31:20
7,Bredasdorp,-34.5322,20.0403,54.48,93,100,3.80,ZA,overcast clouds,2022-08-10 17:32:44
8,Aberdeen,57.1437,-2.0981,82.89,31,43,10.36,GB,scattered clouds,2022-08-10 17:39:44
9,Namibe,-15.1961,12.1522,62.85,88,8,7.63,AO,clear sky,2022-08-10 17:39:44
10,Salalah,17.0151,54.0924,77.09,100,100,4.61,OM,light intensity drizzle,2022-08-10 17:32:55
11,Baykit,61.6700,96.3700,59.31,79,100,1.52,RU,overcast clouds,2022-08-10 17:39:45
12,Teguldet,57.3064,88.1667,61.56,84,24,4.05,RU,few clouds,2022-08-10 17:39:45


In [5]:
# 4a. Determine if there are any empty rows and drop any nulls.
preferred_temperatures_df.count()
clean_df = preferred_temperatures_df.dropna()


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
final_hotel_df = pd.DataFrame(clean_df)
final_hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Decription,Date
0,Hithadhoo,-0.6000,73.0833,82.36,73,64,8.01,MV,broken clouds,2022-08-10 17:39:41
3,Matagami,49.7502,-77.6328,77.61,37,2,12.50,CA,clear sky,2022-08-10 17:39:42
4,Hermanus,-34.4187,19.2345,56.80,88,100,9.42,ZA,overcast clouds,2022-08-10 17:39:43
6,Port Alfred,-33.5906,26.8910,61.83,82,87,4.68,ZA,overcast clouds,2022-08-10 17:31:20
7,Bredasdorp,-34.5322,20.0403,54.48,93,100,3.80,ZA,overcast clouds,2022-08-10 17:32:44


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Decription", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Decription,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.36,broken clouds,-0.6000,73.0833,
3,Matagami,CA,77.61,clear sky,49.7502,-77.6328,
4,Hermanus,ZA,56.80,overcast clouds,-34.4187,19.2345,
6,Port Alfred,ZA,61.83,overcast clouds,-33.5906,26.8910,
7,Bredasdorp,ZA,54.48,overcast clouds,-34.5322,20.0403,
8,Aberdeen,GB,82.89,scattered clouds,57.1437,-2.0981,
9,Namibe,AO,62.85,clear sky,-15.1961,12.1522,
10,Salalah,OM,77.09,light intensity drizzle,17.0151,54.0924,
11,Baykit,RU,59.31,overcast clouds,61.6700,96.3700,
12,Teguldet,RU,61.56,few clouds,57.3064,88.1667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat}, {lng}"
        
        # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?" 

        # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):

        print("Hotel Could Not be Found...Skipping")
        pass
        continue

    

Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping
Hotel Could Not be Found...Skipping


In [41]:
print(hotels)

{'html_attributions': [], 'next_page_token': 'AeJbb3cUoO2P67d6kbJSsDcfFMNGaPHxC4UmIRwoLb49yOr0UgFU8R9Oi6VG6Foyt-MDyikoseV0rtnWOj6y5aC6BrG4JubCoiEYuEvCqwWQx-kk9w23jUPWOcA57zUCauWJKnsGK7ez-xDxtGTJdjirR5gbta8ObounT4uuthD81WuDTqEUsVRf0WUhd5UWSHTgWOUHuRPOBisSDsqI52jXjg324GRmva57bYPqhMEr4lbk2CRerj0zOC6Qs0ya2POUxbUv4N0JC-EmdmpjIPpK-RdkQA1miQVaveuhfQ6A4IHZTWYZ__ZwI4lSZ2UqSehKX8Os_27hgcCKqjrTw8-atDlZDVZtfcp-t8S8yj8kuc5eSH9BlHBrhZ7A_Y-KXRG_GyL-S5SNasKXvLpIFDMvvnjUMfF-QXLw_J_NErqUpUMADEFvexKXowrO', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 20.6887572, 'lng': -105.2308958}, 'viewport': {'northeast': {'lat': 20.6900328802915, 'lng': -105.2296352697085}, 'southwest': {'lat': 20.6873349197085, 'lng': -105.2323332302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'hotel la terminal', 'o

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Decription,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.36,broken clouds,-0.6000,73.0833,Scoop Guest House
3,Matagami,CA,77.61,clear sky,49.7502,-77.6328,Motel Le Caribou
4,Hermanus,ZA,56.80,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
6,Port Alfred,ZA,61.83,overcast clouds,-33.5906,26.8910,The Halyards Hotel
7,Bredasdorp,ZA,54.48,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
...,...,...,...,...,...,...,...
247,Cruzilia,BR,53.08,overcast clouds,-21.8386,-44.8083,Pousada Cruzília
249,Caxito,AO,70.23,clear sky,-8.5785,13.6643,
250,Teruel,ES,78.28,scattered clouds,40.6667,-0.6667,La Parra
251,Dicabisagan,PH,78.42,overcast clouds,17.0818,122.4157,Nancy Montanes Bernardo


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
# 11a. Add a marker layer for each city to the map. 


fig = gmaps.figure(layout={
    'width': '400px',
    'height': '600px'},
    center=(30.0, 31.0), zoom_level=1.5) 

pop_layer = gmaps.marker_layer(locations)


fig.add_layer(pop_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='600px', width='400px'))